In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x:'%.2f'%x)
pd.set_option('display.max_rows', 300)
sns.set(rc={'figure.figsize':(12,8.5)})
import cufflinks as cf

cf.go_offline()


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.express as px

In [ ]:
df = pd.read_csv('renewable.csv')
df_2 = pd.read_csv('final_version.csv')

to_rename = {'country_or_area':'Country',
            'year': 'Year',
             'new_unit':'Unit',
             'new_quantity': 'All production',
             'renewable':'Renewable prod',
             'non_renewable' : 'Non-renewable prod'
                         }

df = df.rename(columns= to_rename)

del df['commodity_transaction']

del df['category']


In [ ]:
def ranking():
    print('Choose year from a range: 1994 to 2014')
    start = ''
    end = ''
    country = ''
    while start == '':
        start = int(input('Enter the initial year: '))
        end = int(input('Enter the final year: '))
        country = input('Enter the country: ')
        if start == end:
            df1 = df[df['Year'] == start]
        df1['Rank'] = df1['Non-renewable prod'].rank(method='dense', ascending = False).astype('int')
        place = df1[df1['Country'].str.lower() == country.lower()]
        num = df1[df1['Country'].str.lower() == country.lower()]['Rank'].values[0]
        mean_c = df1[df1['Rank']<50]
        mean_c = mean_c[['All production','Non-renewable prod','Renewable prod']].mean()
        the_country = place[['Country','All production','Non-renewable prod','Renewable prod']]
        x = pd.DataFrame(mean_c)
        x= x.transpose()
        x.insert(0,'Country','mean')
        show = pd.concat([the_country, x])


        txt = 'Your country: {} was ranked #{} in the year {}.'.format(country,int(num), start)
        print(txt)
        show.iplot(kind = 'bar', x= 'Country', y=['All production','Non-renewable prod','Renewable prod'], 
                   title = 'Country with mean of top 50 country of choosen year')


        else:
            df1 = df[(df['Year'] >= start) & (df['Year'] <= end)]
            df_rank = df1.groupby('Country').agg({'All production':'mean', 'Non-renewable prod':'mean', 'Renewable prod':'mean'}).reset_index()
            year = df1.groupby('Year').agg({'All production':'mean', 'Non-renewable prod':'mean', 'Renewable prod':'mean'}).reset_index()

            df_rank['Rank'] = df_rank['Non-renewable prod'].rank(method='dense', ascending = False).astype('int')
            place = df_rank[df_rank['Country'].str.lower() == country.lower()]
            num = df_rank[df_rank['Country'].str.lower() == country.lower()]['Rank'].values[0]


            three_countries = df_rank[(df_rank['Rank']>=num-1) & (df_rank['Rank']<=num+1)].sort_values('Rank')
            countries = three_countries.values[:,0]
            df1c = df1[df1['Country'].isin(countries)]
            # three_countries['%_renewable'] = (three_countries['renewable']/ three_countries['new_quantity'])*100
            # three_countries['%_non_renewable'] = (three_countries['non_renewable']/ three_countries['new_quantity'])*100
            txt = 'Your chosen country {} was ranked #{} in years {} - {}.'.format(country,int(num),start, end)
            print(txt, '\n')
            print(three_countries[['Country','Rank','All production', 'Non-renewable prod', 'Renewable prod']], '\n')
            three_countries[['Country','Rank','All production', 'Non-renewable prod', 'Renewable prod']]
            print('Average of top 50 production in those years:')
            mena = df_rank[df_rank['Rank']<=50]
            mena = mena[['All production', 'Non-renewable prod', 'Renewable prod']]
            x = mena.mean()
            print(x.transpose())
            


            fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10,10))
            ax1 = axes[0]
            ax2 = axes[1]
            ax3 = axes[2]
            #ax4 = axes[3]



            sns.lineplot(data = df1c, x = 'Year', y = 'All production',ax=ax1, hue = 'Country', )
            sns.lineplot(data = year, x = 'Year', y = 'All production',ax=ax1, color = 'black', label = 'Total production')
            ax1.set_title('All production',loc = 'center')
            ax1.set_xlabel('')
            ax1.set_ylabel('Electircity production [TWh]')
            ax1.set_xticklabels([])
            ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
            ax1.set_label('Country')

            sns.lineplot(data = df1c, x = 'Year', y = 'Renewable prod',ax=ax2, hue = 'Country' )
            sns.lineplot(data = year, x = 'Year', y = 'Renewable prod',ax=ax2, color = 'black', label ='Renewable' )
            ax2.set_title('Renewable production', loc = 'center')
            ax2.set_xlabel('')
            ax2.set_ylabel('Electircity production [TWh]')
            ax2.set_xticklabels([])
            ax2.xaxis.set_major_locator(MaxNLocator(integer=True))

            sns.lineplot(data = df1c, x = 'Year', y = 'Non-renewable prod',ax=ax3, hue = 'Country' )
            sns.lineplot(data = year, x = 'Year', y = 'Non-renewable prod',ax=ax3, color = 'black', label ='Non renewable production')
            ax3.set_title('Non renewable production', loc = 'center')
            # ax3.set_xlabel('')
            ax3.set_ylabel('Electircity production [TWh]')
            ax3.set_xlabel('Year')
            #ax3.set_xticklabels([])
            ax3.xaxis.set_major_locator(MaxNLocator(integer=True))

            # sns.lineplot(data = year, x = 'year', y = 'renewable',ax=ax4, color = 'orange', label ='Renewable' )
            # sns.lineplot(data = year, x = 'year', y = 'non_renewable',ax=ax4, color = 'pink', label ='Non renewable')
            # sns.lineplot(data = year, x = 'year', y = 'new_quantity',ax=ax4, color = 'black', label = 'Total production')
            # ax4.set_title('Average production in selected years', loc = 'right')
            # ax4.set_xlabel('Year')
            # ax4.set_ylabel('[Terawatt-hour]')
            # ax4.xaxis.set_major_locator(MaxNLocator(integer=True))
            fig.suptitle('Energy production', fontsize=20)
            plt.show()
            
            data_list = ['Electricity - total thermal production',
                 'Electricity - total nuclear production', 'Electricity - total geothermal production',
                 'Electricity - total solar production',
                 'Electricity - total tide, wave and hydro production', 'Electricity - total wind production',
                 'From chemical heat – Autoproducer','From chemical heat – Main activity',
                 'From other sources – Autoproducer','From other sources – Main activity']
            
            df2 = df_2[df_2['commodity_transaction'].isin(data_list)]
            filtered=(df2.loc[df2['country_or_area'].str.lower() == country.lower()])
            years = [start, end]
            filtered2=(filtered.loc[filtered.year.isin(years)])
            sum_country = filtered2.groupby(['category', 'year'])['new_quantity'].sum().unstack().reset_index()
            sum_country['diff TWh'] = (sum_country.loc[:,end] - sum_country.loc[:,start])  # probowałem odwołać się do zmiennej min/max year nie wiem jak to zrobić
            sum_country['diff %'] = (sum_country.loc[:,'diff TWh'] / sum_country.loc[:,start]*100)
            return sum_country

